In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.corpus import stopwords
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = stopwords.words('english') 
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azizb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\azizb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

84 documents
15 classes ['Asthma symptoms', 'Consultation', 'Depression symptoms', 'Diabetes symptoms', 'age', 'asthma prevention', 'common cold prevention', 'common cold symptoms', 'depression prevention', 'diabetes prevention', 'fever prevention', 'fever symptoms', 'goodbye', 'greetings', 'name']
107 unique lemmatized words ["'s", '(', ')', '?', 'ache', 'age', 'anxiety', 'appetite', 'asthma', 'available', 'blurry', 'body', 'breath', 'buy', 'bye', 'call', 'cao', 'change', 'chest', 'chill', 'cold', 'common', 'congestion', 'consultation', 'contact', 'cough', 'coughing', 'cya', 'day', 'death', 'dehydration', 'depression', 'diabetes', 'difficulty', 'doctor', 'drink', 'eat', 'extreme', 'fatigue', 'feeling', 'fever', 'flu', 'frequent', 'general', 'generally', 'getting', 'give', 'going', 'good', 'goodbye', 'greeting', 'headache', 'hello', 'help', 'hey', 'hi', 'hopeless', 'how', 'hunger', 'i', 'increased', 'interest', 'irritability', 'keep', 'later', 'looking', 'loss', 'lost', 'low-grade', 'm

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-8-5117620404a0>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [24]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
17/17 [==============================] - 1s 9ms/step - loss: 2.7082 - accuracy: 0.0238
Epoch 2/200
17/17 [==============================] - 0s 8ms/step - loss: 2.6795 - accuracy: 0.0952
Epoch 3/200
17/17 [==============================] - 0s 8ms/step - loss: 2.5745 - accuracy: 0.2143
Epoch 4/200
17/17 [==============================] - 0s 8ms/step - loss: 2.5227 - accuracy: 0.1548
Epoch 5/200
17/17 [==============================] - 0s 10ms/step - loss: 2.5091 - accuracy: 0.2738
Epoch 6/200
17/17 [==============================] - 0s 9ms/step - loss: 2.4213 - accuracy: 0.2024
Epoch 7/200
17/17 [==============================] - 0s 8ms/step - loss: 2.3748 - accuracy: 0.2262
Epoch 8/200
17/17 [==============================] - 0s 8ms/step - loss: 2.2728 - accuracy: 0.2976
Epoch 9/200
17/17 [==============================] - 0s 8ms/step - loss: 2.0911 - accuracy: 0.3333
Epoch 10/200
17/17 [==============================] - 0s 8ms/step - loss: 2.0403 - accuracy: 0.2738
Epoch 11

In [25]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [26]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [27]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [50]:
chatbot_response("it's really cold")

'It seems that you are suffering from fever'